In [3]:
from discog_func import *

token = 'eGkJSFluvBIeQCEmsueVEMZIabFbDLaPFIDcKFkg'
artist = 'Myron Walden'

In [4]:
collabs = discog(artist,token)
top_five = collabs[:5]

346616


In [5]:
top_five

[(346616,
  [(u'Hypnosis', 1996),
   (u'Brian Blade Fellowship', 1998),
   (u'Perceptual', 2000),
   (u'Premonition', 2000),
   (u'Salt', 2003),
   (u'Treats For The Nightwalker', 2003),
   (u'Season Of Changes', 2008),
   (u'On The Real Side', 2008),
   (u'Landmarks', 2014)]),
 (252331,
  [(u'Brian Blade Fellowship', 1998),
   (u'Perceptual', 2000),
   (u'Salt', 2003),
   (u'Season Of Changes', 2008),
   (u'Landmarks', 2014)]),
 (510174,
  [(u'Brian Blade Fellowship', 1998),
   (u'Perceptual', 2000),
   (u'Salt', 2003),
   (u'Season Of Changes', 2008),
   (u'Landmarks', 2014)]),
 (510176,
  [(u'Brian Blade Fellowship', 1998),
   (u'Perceptual', 2000),
   (u'Season Of Changes', 2008),
   (u'Landmarks', 2014)]),
 (460771,
  [(u'Brian Blade Fellowship', 1998),
   (u'Perceptual', 2000),
   (u'Season Of Changes', 2008)])]

In [9]:
# import discogs_client
# d = discogs_client.Client('ExampleApplication/0.2',user_token=token)
# a_id = d.search(artist, type='artist')[0].id

In [ ]:
node_trace, edge_trace = construct_g(artist,top_five)

In [306]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title=artist + "'s Top Five Collaborators", #'<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations= annots,
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jglub212/0 or inside your plot.ly account where it is named 'networkx'
